Bu bölümde, Pandas'da verileri birleştirmeye yarayan basit bir fonksiyon olan `pd.concat()` fonksiyonuyla Serie ve DataFrame'leri birleştirmeyi öğreneceğiz.

Çok kullanmamız gerekeceğinden öncelikle DataFrame oluşturan bir fonksiyon tanımlayalım.

In [1]:
import numpy as np
import pandas as pd

def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABC', [1, 2, 3])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3


### Hatırlayalım: Numpy'da Veri Birleştirme

Numpy dizilerini birleştirmeyi [Numpy Dizileri](#) bölümünde görmüştük. `np.concatenate()` fonksiyonu ile yapılıyordu. İlk parametre olarak birleştirilecek dizileri, isteğe bağlı olarak da `axis` parametresi ile hangi eksende birleştirileceğini belirtiyorduk.

In [2]:
a = [1, 2]
b = [3, 4]
np.concatenate([a, b])

array([1, 2, 3, 4])

## pd.concat ile Basit Birleştirme İşlemi

`pd.concat()` fonksiyonu kullanım açısından `np.concatenate()` fonksiyonuna oldukça benzerdir.

In [3]:
a = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
b = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([a, b])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

Birleştirme işlemini çok boyutlu dizilerde de yapabilirsiniz.

In [4]:
a = make_df('AB', [1, 2])
a

,A,B
1,A1,B1
2,A2,B2


In [5]:
b = make_df('AB', [3, 4])
b

,A,B
3,A3,B3
4,A4,B4


In [6]:
pd.concat([a, b])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


`axis` parametresi ile hangi eksende birleştireceğini belirtebilirsiniz

### Index'lerin Tekrarlanması

`pd.concat()` fonksiyonunun `np.concatenate()` fonksiyonundan önemli bir farkı tekrarlanmış indexleri koruması, değiştirmemesidir.


In [7]:
a = make_df('AB', [0, 1])
a

,A,B
0,A0,B0
1,A1,B1


In [8]:
b = make_df('AB', [0, 1])
b

,A,B
0,A0,B0
1,A1,B1


In [9]:
pd.concat([a, b])

,A,B
0,A0,B0
1,A1,B1
0,A0,B0
1,A1,B1


Görüldüğü üzere 0 ve 1 indexleri tekrar ediyor. Bu DataFrame için sorun oluşturmasa da istenmeyen bir durumdur. Bu durumu engellemenin birkaç yolu bulunuyor.

1. Hata olarak yakalama

Tekrar eden index bulunuyorsa hata vermesini sağlayabilirsiniz. Bunun için `verify_integrity` parametresi bulunuyor. `True` yaparsanız, çakışan index olduğunda hata verecektir. try catch içine alarak tekrarlanan index olup olmadığını anlayabilirsiniz.

In [10]:
try:
    pd.concat([a, b], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


2. Indexleri korumadan birleştirme

Bazı verilerde indexlerin bir önemi yoktur. Bu durumda `ignore_flag` parametresini `True` yaparak indexlerin dikkate alınmadan sadece verilerin birleştirilmesini sağlayabilirsiniz.

In [11]:
pd.concat([a, b], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A0,B0
3,A1,B1


3. MultiIndex yapma

`keys` parametresi ile verilere ad vererek MultiIndex yapısıyla birleştirebilirsiniz. MultiIndex yapısını [bir önceki bölümde](#) incelemiştik.

In [12]:
pd.concat([a, b], keys=['dizi1', 'dizi2'])

A   B
dizi1 0  A0  B0
      1  A1  B1
dizi2 0  A0  B0
      1  A1  B1

### Verileri Birbirine Bağlama

Verilerimizin aynı veya farklı indexleri olabileceği gibi bu verilerin kesiştikleri ve ayrı oldukları yerler de olabilir. Aşağıda iki DataFrame görüyorsunuz. B ve C sütunları ikisinde de varken A ve D için aynısını diyemeyiz.

In [13]:
a = make_df('ABC', [1, 2])
a

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [14]:
b = make_df('BCD', [3, 4])
b

,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [15]:
pd.concat([a, b])

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


? burda kaldım
Gördüğünüz gibi veri koyamadığı yerlere NaN koydu. Bunun olmaması için `join` ve `join_axes` parametrelerimiz var.

Öntanımlı olarak join değerimiz 'outer' olarak tanımlıdır ve iki verinin birleşimi anlamına gelir. 'inner' olarak ayarlarsak iki verinin kesişimini al anlamına gelir. Yani şimdiki durum için A ve D sütunlarını alma.

In [16]:
pd.concat([a, b], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


?join_axes için futurewarning verdi. reindex kullan diyor.

### append() Metodu

İki veriyi birleştirme çok sık kullanıldığından Serie ve DataFrame verileri append() adında metod bulundurur. Parametre olarak kendisiyle birleştirilecek veriyi alır ve birleşmiş halini döndürür.

In [17]:
a.append(b)

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Bu durumda `pd.concat([a, b])` ile aynı işlemi yapar.

Yazımı oldukça kolaylaştırsa da buffer kullanımından dolayı çok dizi birleştirmek için önerilmez.

In [18]:
c = make_df('AB', [1, 2])
d = make_df('CD', [3, 4])
%timeit a.append(b).append(c).append(d)
%timeit pd.concat([a, b, c, d])

2.01 ms ± 30.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.18 ms ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
